<a href="https://colab.research.google.com/github/amscally-dsci/dd-test-public/blob/main/AMS_Try_updated_Copy_of_3c_Feature_Selection_end.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Feature Selection


Feature selection is the process of determining what features to include in your model. Why do we need feature selection?


*   To prevent overfitting - models with more features are more prone to overfitting
*   To simplify our model - simpler models are easier to interpret and debug
*   To reduce computation time - models run faster with less features


There are several ways we can perform feature selection
1. Filter methods
2. Wrapper methods such as forward selection (there are known issues with some of these methods)
3. Regularization methods (these are discussed in more detail in another lecture)


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler

%%capture
%%bash
apt-get update
apt-get install -y tree

SyntaxError: invalid syntax (<ipython-input-126-1344052417>, line 12)

## Read in Data

In [ ]:
# Read in data
url = "https://ddc-datascience.s3.amazonaws.com/Projects/Project.2-Housing/Data/Housing.Data.csv "
housing = pd.read_csv(url)
housing.head()

,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,905101070,20,RL,62.0,14299,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,7,2007,WD,Normal,115400
1,905101330,90,RL,72.0,10791,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,Shed,500,10,2006,WD,Normal,90000
2,903454090,50,RM,50.0,9000,Pave,NaN,Reg,Bnk,AllPub,...,0,NaN,NaN,NaN,0,12,2007,WD,Normal,141000
3,533244030,60,FV,68.0,7379,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,254000
4,909252020,70,RL,60.0,7200,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,4,2009,WD,Normal,155000


In [ ]:
housing.shape

(2637, 81)

In [ ]:
type(housing.shape)

tuple

In [ ]:
rows, columns = housing.shape
{
  "Rows": rows,
  "Columns" : columns
}

{'Rows': 2637, 'Columns': 81}

In [ ]:
housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2637 entries, 0 to 2636
Data columns (total 81 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   PID              2637 non-null   int64  
 1   MS SubClass      2637 non-null   int64  
 2   MS Zoning        2637 non-null   object 
 3   Lot Frontage     2188 non-null   float64
 4   Lot Area         2637 non-null   int64  
 5   Street           2637 non-null   object 
 6   Alley            180 non-null    object 
 7   Lot Shape        2637 non-null   object 
 8   Land Contour     2637 non-null   object 
 9   Utilities        2637 non-null   object 
 10  Lot Config       2637 non-null   object 
 11  Land Slope       2637 non-null   object 
 12  Neighborhood     2637 non-null   object 
 13  Condition 1      2637 non-null   object 
 14  Condition 2      2637 non-null   object 
 15  Bldg Type        2637 non-null   object 
 16  House Style      2637 non-null   object 
 17  Overall Qual  

In [ ]:
# data set overview

# define df shape and size
housing_shape=housing.shape
rows=housing_shape[0]
columns=housing_shape[1]
housing_size=housing.size

# Define df characteristics
total_nulls=housing.isnull().sum().sum()
col_null_counts = housing.isnull().sum()

print(f"""The Original Housing Data Set Dimensions
      Total Elements  {housing_size:,}
      Total Rows      {rows}
      Total Columns   {columns}""")


print(f"""The Original Housing Data Set Characteristics
      Total Nulls in Data Frame     {total_nulls}""")


The Original Housing Data Set Dimensions
      Total Elements  213,597
      Total Rows      2637
      Total Columns   81
The Original Housing Data Set Characteristics
      Total Nulls in Data Frame     14208


## Data Cleaning

In [ ]:
# make copy of original data frame before cleaning
housing_backup=housing.copy()
housing_original=housing.copy()
# still use housing due to metadata later on
housing.head()

,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,905101070,20,RL,62.0,14299,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,7,2007,WD,Normal,115400
1,905101330,90,RL,72.0,10791,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,Shed,500,10,2006,WD,Normal,90000
2,903454090,50,RM,50.0,9000,Pave,NaN,Reg,Bnk,AllPub,...,0,NaN,NaN,NaN,0,12,2007,WD,Normal,141000
3,533244030,60,FV,68.0,7379,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,254000
4,909252020,70,RL,60.0,7200,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,4,2009,WD,Normal,155000


In [ ]:
housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2637 entries, 0 to 2636
Data columns (total 81 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   PID              2637 non-null   int64  
 1   MS SubClass      2637 non-null   int64  
 2   MS Zoning        2637 non-null   object 
 3   Lot Frontage     2188 non-null   float64
 4   Lot Area         2637 non-null   int64  
 5   Street           2637 non-null   object 
 6   Alley            180 non-null    object 
 7   Lot Shape        2637 non-null   object 
 8   Land Contour     2637 non-null   object 
 9   Utilities        2637 non-null   object 
 10  Lot Config       2637 non-null   object 
 11  Land Slope       2637 non-null   object 
 12  Neighborhood     2637 non-null   object 
 13  Condition 1      2637 non-null   object 
 14  Condition 2      2637 non-null   object 
 15  Bldg Type        2637 non-null   object 
 16  House Style      2637 non-null   object 
 17  Overall Qual  

Notice `.info()` provides five kinds of information:
- the number of rows and columns
- the number of rows of each column that are not-nulls
- the data type of the column
- summary counts of column data types
- the amount of memory used by the Data Frame

How can I extract that information to work with it?  For example, how can I use the data from `.info()` to list all columns that have nulls?

In [ ]:
# Only shows numeric entries which are floats and ints total 38 27+11
housing.describe()

,PID,MS SubClass,Lot Frontage,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,BsmtFin SF 1,...,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,SalePrice
count,2.637000e+03,2637.000000,2188.000000,2637.000000,2637.000000,2637.000000,2637.000000,2637.000000,2614.000000,2636.000000,...,2637.000000,2637.000000,2637.000000,2637.000000,2637.000000,2637.000000,2637.000000,2637.000000,2637.000000,2637.000000
mean,7.141301e+08,57.349261,69.166819,10044.694729,6.097459,5.569966,1971.288586,1984.202882,101.887911,438.441199,...,94.305271,46.984452,22.813424,2.368601,15.775123,2.130072,42.014031,6.243838,2007.795601,179986.230186
std,1.887527e+08,42.499091,23.356779,6742.549521,1.411522,1.118262,30.306986,20.913077,179.578232,449.602326,...,126.927272,66.564333,61.177638,23.160600,55.783751,35.140140,393.158781,2.722093,1.306403,78309.251522
min,5.263011e+08,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,12789.000000
25%,5.284770e+08,20.000000,58.000000,7436.000000,5.000000,5.000000,1954.000000,1965.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,2007.000000,129500.000000
50%,5.354530e+08,50.000000,68.000000,9450.000000,6.000000,5.000000,1973.000000,1993.000000,0.000000,368.000000,...,0.000000,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,160000.000000
75%,9.071870e+08,70.000000,80.000000,11526.000000,7.000000,6.000000,2001.000000,2004.000000,164.000000,732.000000,...,168.000000,70.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,213000.000000
max,1.007100e+09,190.000000,313.000000,164660.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,1424.000000,742.000000,584.000000,407.000000,576.000000,800.000000,12500.000000,12.000000,2010.000000,745000.000000


In [ ]:
housing.describe().columns

Index(['PID', 'MS SubClass', 'Lot Frontage', 'Lot Area', 'Overall Qual',
       'Overall Cond', 'Year Built', 'Year Remod/Add', 'Mas Vnr Area',
       'BsmtFin SF 1', 'BsmtFin SF 2', 'Bsmt Unf SF', 'Total Bsmt SF',
       '1st Flr SF', '2nd Flr SF', 'Low Qual Fin SF', 'Gr Liv Area',
       'Bsmt Full Bath', 'Bsmt Half Bath', 'Full Bath', 'Half Bath',
       'Bedroom AbvGr', 'Kitchen AbvGr', 'TotRms AbvGrd', 'Fireplaces',
       'Garage Yr Blt', 'Garage Cars', 'Garage Area', 'Wood Deck SF',
       'Open Porch SF', 'Enclosed Porch', '3Ssn Porch', 'Screen Porch',
       'Pool Area', 'Misc Val', 'Mo Sold', 'Yr Sold', 'SalePrice'],
      dtype='object')

In [ ]:
housing.describe().columns.size

38

In [ ]:
housing.describe( include = "all" ).columns.size

81

In [ ]:
(
  housing
  .describe( include = "all" )
  .transpose()
  .astype({"count": int})
)

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
PID,2637,NaN,NaN,NaN,714130147.70383,188752674.750322,526301100.0,528477010.0,535453040.0,907187010.0,1007100110.0
MS SubClass,2637,NaN,NaN,NaN,57.349261,42.499091,20.0,20.0,50.0,70.0,190.0
MS Zoning,2637,7,RL,2043,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Lot Frontage,2188,NaN,NaN,NaN,69.166819,23.356779,21.0,58.0,68.0,80.0,313.0
Lot Area,2637,NaN,NaN,NaN,10044.694729,6742.549521,1300.0,7436.0,9450.0,11526.0,164660.0
...,...,...,...,...,...,...,...,...,...,...,...
Mo Sold,2637,NaN,NaN,NaN,6.243838,2.722093,1.0,4.0,6.0,8.0,12.0
Yr Sold,2637,NaN,NaN,NaN,2007.795601,1.306403,2006.0,2007.0,2008.0,2009.0,2010.0
Sale Type,2637,10,WD,2286,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sale Condition,2637,6,Normal,2166,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# how can/did you discover if there are any unique identifiers and how would/did you handle them?
non=housing.nunique().sort_values(ascending = False)
non

,0
PID,2637
Lot Area,1799
Gr Liv Area,1216
Bsmt Unf SF,1080
1st Flr SF,1022
...,...
Bsmt Half Bath,3
Paved Drive,3
Street,2
Alley,2


### Nulls


In [ ]:
housing.isnull()

,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,False,False,False,False,False,False,True,False,False,False,...,False,True,False,True,False,False,False,False,False,False
1,False,False,False,False,False,False,True,False,False,False,...,False,True,True,False,False,False,False,False,False,False
2,False,False,False,False,False,False,True,False,False,False,...,False,True,True,True,False,False,False,False,False,False
3,False,False,False,False,False,False,True,False,False,False,...,False,True,True,True,False,False,False,False,False,False
4,False,False,False,False,False,False,True,False,False,False,...,False,True,False,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2632,False,False,False,False,False,False,True,False,False,False,...,False,True,False,False,False,False,False,False,False,False
2633,False,False,False,False,False,False,True,False,False,False,...,False,True,True,True,False,False,False,False,False,False
2634,False,False,False,False,False,False,True,False,False,False,...,False,True,True,True,False,False,False,False,False,False
2635,False,False,False,False,False,False,True,False,False,False,...,False,True,False,True,False,False,False,False,False,False


In [ ]:
housing.isnull().sum()

,0
PID,0
MS SubClass,0
MS Zoning,0
Lot Frontage,449
Lot Area,0
...,...
Mo Sold,0
Yr Sold,0
Sale Type,0
Sale Condition,0


In [ ]:
# percent nulls     count nulls/2637
nulls = housing.isnull().sum()
filter = nulls > 0
( nulls[ filter ].sort_values( ascending = False ) / housing_original.shape[0] * 100 ).round(1)

,0
Pool QC,99.6
Misc Feature,96.4
Alley,93.2
Fence,80.0
Mas Vnr Type,60.9
Fireplace Qu,48.7
Lot Frontage,17.0
Garage Cond,5.6
Garage Yr Blt,5.6
Garage Finish,5.6


In [ ]:
# data types-feature by data type
housing.dtypes

,0
PID,int64
MS SubClass,int64
MS Zoning,object
Lot Frontage,float64
Lot Area,int64
...,...
Mo Sold,int64
Yr Sold,int64
Sale Type,object
Sale Condition,object


In [ ]:
# memory usage
housing.memory_usage(deep=False, index = False)

,0
PID,21096
MS SubClass,21096
MS Zoning,21096
Lot Frontage,21096
Lot Area,21096
...,...
Mo Sold,21096
Yr Sold,21096
Sale Type,21096
Sale Condition,21096


In [ ]:
housing.memory_usage(deep=False, index = False).sum()

np.int64(1708776)

In [ ]:
housing.memory_usage(deep=True, index = False).sum()

np.int64(7303051)

In [ ]:
# assign current data frame to generic name
df=housing
#Building a data frame to contain the metadata for the df for ease of info extraction.
df_info=pd.concat([
    pd.DataFrame([ df.isna().sum().to_dict() ], index = ["Nulls"] ).transpose().astype( { "Nulls": int } ),
    pd.DataFrame( [ df.dtypes.to_dict() ], index = [ "Data_Types"] ).transpose().astype( { "Data_Types": 'category' }),
    pd.DataFrame( [ df.memory_usage(deep=True, index = False).to_dict() ], index = ["Memory"] ).transpose(),
    df.describe( include = "all" ).transpose().astype( { "count": int } ).rename( columns = { "50%" : "median"} ),
    ], axis = "columns")
df_info["IRQ"] = df_info["75%"] - df_info["25%"]
df_info["range"] = df_info["max"] - df_info["min"]
df_info["sum"] = df_info["mean"] * df_info["count"]
df_info.insert( 1, "Nulls_pct" , round( df_info["Nulls"] / df_info.shape[0], 2) )

# Display some info from the meta data
print( f"Memory: { (df_info['Memory'].sum() / 1000 / 1000 ):_} MB" )
print( df_info["Data_Types"].value_counts().to_json() )
df_info.sort_values(by = ["Data_Types", "max"], ascending=[1,0])


Memory: 7.303051 MB
{"object":43,"int64":27,"float64":11}


,Nulls,Nulls_pct,Data_Types,Memory,count,unique,top,freq,mean,std,min,25%,median,75%,max,IRQ,range,sum
PID,0,0.00,int64,21096,2637,NaN,NaN,NaN,714130147.70383,188752674.750322,526301100.0,528477010.0,535453040.0,907187010.0,1007100110.0,378710000.0,480799010.0,1883161199495.0
SalePrice,0,0.00,int64,21096,2637,NaN,NaN,NaN,179986.230186,78309.251522,12789.0,129500.0,160000.0,213000.0,745000.0,83500.0,732211.0,474623689.0
Lot Area,0,0.00,int64,21096,2637,NaN,NaN,NaN,10044.694729,6742.549521,1300.0,7436.0,9450.0,11526.0,164660.0,4090.0,163360.0,26487860.0
Misc Val,0,0.00,int64,21096,2637,NaN,NaN,NaN,42.014031,393.158781,0.0,0.0,0.0,0.0,12500.0,0.0,12500.0,110791.0
Gr Liv Area,0,0.00,int64,21096,2637,NaN,NaN,NaN,1496.98521,495.209631,334.0,1128.0,1441.0,1740.0,5642.0,612.0,5308.0,3947550.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Pool QC,2626,32.42,object,84681,11,4,Gd,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Fence,2109,26.04,object,100110,528,4,MnPrv,306,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Misc Feature,2541,31.37,object,87168,96,4,Shed,87,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sale Type,0,0.00,object,158288,2637,10,WD,2286,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Note**: for later?
- remove columns with 20% or more of null values


- remove rows with 5% or fewer null values




In [ ]:
for dt in df_info["Data_Types"].unique():
  print(dt)

int64
object
float64


In [ ]:
df_info["Data_Types"].value_counts()


,count
Data_Types,
object,43
int64,27
float64,11


In [ ]:
for dt in df_info["Data_Types"].unique():
  filter = df_info["Data_Types"] == dt
  cols = df_info[ filter ].index
  housing[ cols ].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2637 entries, 0 to 2636
Data columns (total 27 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   PID              2637 non-null   int64
 1   MS SubClass      2637 non-null   int64
 2   Lot Area         2637 non-null   int64
 3   Overall Qual     2637 non-null   int64
 4   Overall Cond     2637 non-null   int64
 5   Year Built       2637 non-null   int64
 6   Year Remod/Add   2637 non-null   int64
 7   1st Flr SF       2637 non-null   int64
 8   2nd Flr SF       2637 non-null   int64
 9   Low Qual Fin SF  2637 non-null   int64
 10  Gr Liv Area      2637 non-null   int64
 11  Full Bath        2637 non-null   int64
 12  Half Bath        2637 non-null   int64
 13  Bedroom AbvGr    2637 non-null   int64
 14  Kitchen AbvGr    2637 non-null   int64
 15  TotRms AbvGrd    2637 non-null   int64
 16  Fireplaces       2637 non-null   int64
 17  Wood Deck SF     2637 non-null   int64
 18  Open Por

In [ ]:
# finding unique values by data type not sure...perhaps to create Identifier List?
filter = df_info["Data_Types"] == "int64"
cols = df_info[ filter ].index
housing[ cols ].nunique().sort_values(ascending = False) / 2637 * 100

,0
PID,100.000000
Lot Area,68.221464
Gr Liv Area,46.113007
1st Flr SF,38.756162
SalePrice,37.277209
2nd Flr SF,22.335988
Wood Deck SF,13.917330
Open Porch SF,9.025408
Enclosed Porch,6.636329
Year Built,4.474782


In [ ]:
filter = df_info["Data_Types"] == "float64"
cols = df_info[ filter ].index
housing[ cols ].nunique().sort_values(ascending = False) / 2637 * 100

,0
Bsmt Unf SF,40.955631
Total Bsmt SF,37.959803
BsmtFin SF 1,35.456959
Garage Area,21.843003
Mas Vnr Area,16.116799
BsmtFin SF 2,9.594236
Lot Frontage,4.778157
Garage Yr Blt,3.868032
Garage Cars,0.227531
Bsmt Full Bath,0.151688


In [ ]:
filter = df_info["Data_Types"] == "object"
cols = df_info[ filter ].index
housing[ cols ].nunique().sort_values(ascending = False) / 2637 * 100

,0
Neighborhood,1.061813
Exterior 2nd,0.644672
Exterior 1st,0.606750
Sale Type,0.379219
Condition 1,0.341297
House Style,0.303375
Functional,0.303375
Roof Matl,0.303375
Condition 2,0.303375
MS Zoning,0.265453


Nex Step is Automoate a blank data frame_2nd pass
from Lecture 1d module 3

## Automate: second pass

### Blank data frame

In [ ]:
metatdata_df = pd.DataFrame()
metatdata_df

""


### Append data type info

In [ ]:
metatdata_df["Data_types"] = housing.dtypes
metatdata_df

,Data_types
PID,int64
MS SubClass,int64
MS Zoning,object
Lot Frontage,float64
Lot Area,int64
...,...
Mo Sold,int64
Yr Sold,int64
Sale Type,object
Sale Condition,object


### Append Null stats

In [ ]:
metatdata_df["Nulls"] = housing.isnull().sum()
metatdata_df

,Data_types,Nulls
PID,int64,0
MS SubClass,int64,0
MS Zoning,object,0
Lot Frontage,float64,449
Lot Area,int64,0
...,...,...
Mo Sold,int64,0
Yr Sold,int64,0
Sale Type,object,0
Sale Condition,object,0


In [ ]:
metatdata_df["Nulls_pct"] = ( metatdata_df["Nulls"] / housing.shape[0] * 100 ).round(1)
metatdata_df

,Data_types,Nulls,Nulls_pct
PID,int64,0,0.0
MS SubClass,int64,0,0.0
MS Zoning,object,0,0.0
Lot Frontage,float64,449,17.0
Lot Area,int64,0,0.0
...,...,...,...
Mo Sold,int64,0,0.0
Yr Sold,int64,0,0.0
Sale Type,object,0,0.0
Sale Condition,object,0,0.0


### Append memory usage

In [ ]:
metatdata_df["Memory"] = housing.memory_usage( deep = True)
metatdata_df

,Data_types,Nulls,Nulls_pct,Memory
PID,int64,0,0.0,21096
MS SubClass,int64,0,0.0,21096
MS Zoning,object,0,0.0,155728
Lot Frontage,float64,449,17.0,21096
Lot Area,int64,0,0.0,21096
...,...,...,...,...
Mo Sold,int64,0,0.0,21096
Yr Sold,int64,0,0.0,21096
Sale Type,object,0,0.0,158288
Sale Condition,object,0,0.0,166537


### Append unique counts and percentage


In [ ]:
metatdata_df["NUnique"] = housing.nunique()
metatdata_df


,Data_types,Nulls,Nulls_pct,Memory,NUnique
PID,int64,0,0.0,21096,2637
MS SubClass,int64,0,0.0,21096,16
MS Zoning,object,0,0.0,155728,7
Lot Frontage,float64,449,17.0,21096,126
Lot Area,int64,0,0.0,21096,1799
...,...,...,...,...,...
Mo Sold,int64,0,0.0,21096,12
Yr Sold,int64,0,0.0,21096,5
Sale Type,object,0,0.0,158288,10
Sale Condition,object,0,0.0,166537,6


In [ ]:
metatdata_df["NUnique_pct"] = (housing.nunique()/len(housing)*100).round(1)
metatdata_df


,Data_types,Nulls,Nulls_pct,Memory,NUnique,NUnique_pct
PID,int64,0,0.0,21096,2637,100.0
MS SubClass,int64,0,0.0,21096,16,0.6
MS Zoning,object,0,0.0,155728,7,0.3
Lot Frontage,float64,449,17.0,21096,126,4.8
Lot Area,int64,0,0.0,21096,1799,68.2
...,...,...,...,...,...,...
Mo Sold,int64,0,0.0,21096,12,0.5
Yr Sold,int64,0,0.0,21096,5,0.2
Sale Type,object,0,0.0,158288,10,0.4
Sale Condition,object,0,0.0,166537,6,0.2


### Append descriptive stats

Use .join() instead of .concatenate().  This ensures the named index is used to match values.

In [ ]:
metatdata_df = metatdata_df.join( housing.describe( include = "all" ).transpose() )
metatdata_df

,Data_types,Nulls,Nulls_pct,Memory,NUnique,NUnique_pct,count,unique,top,freq,mean,std,min,25%,50%,75%,max
PID,int64,0,0.0,21096,2637,100.0,2637.0,NaN,NaN,NaN,714130147.70383,188752674.750322,526301100.0,528477010.0,535453040.0,907187010.0,1007100110.0
MS SubClass,int64,0,0.0,21096,16,0.6,2637.0,NaN,NaN,NaN,57.349261,42.499091,20.0,20.0,50.0,70.0,190.0
MS Zoning,object,0,0.0,155728,7,0.3,2637,7,RL,2043,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Lot Frontage,float64,449,17.0,21096,126,4.8,2188.0,NaN,NaN,NaN,69.166819,23.356779,21.0,58.0,68.0,80.0,313.0
Lot Area,int64,0,0.0,21096,1799,68.2,2637.0,NaN,NaN,NaN,10044.694729,6742.549521,1300.0,7436.0,9450.0,11526.0,164660.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Mo Sold,int64,0,0.0,21096,12,0.5,2637.0,NaN,NaN,NaN,6.243838,2.722093,1.0,4.0,6.0,8.0,12.0
Yr Sold,int64,0,0.0,21096,5,0.2,2637.0,NaN,NaN,NaN,2007.795601,1.306403,2006.0,2007.0,2008.0,2009.0,2010.0
Sale Type,object,0,0.0,158288,10,0.4,2637,10,WD,2286,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sale Condition,object,0,0.0,166537,6,0.2,2637,6,Normal,2166,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Adjust format of data frame

In [ ]:
metatdata_df = metatdata_df.astype( {"count": int }).rename( columns = {"50%": "median"})
metatdata_df

,Data_types,Nulls,Nulls_pct,Memory,NUnique,NUnique_pct,count,unique,top,freq,mean,std,min,25%,median,75%,max
PID,int64,0,0.0,21096,2637,100.0,2637,NaN,NaN,NaN,714130147.70383,188752674.750322,526301100.0,528477010.0,535453040.0,907187010.0,1007100110.0
MS SubClass,int64,0,0.0,21096,16,0.6,2637,NaN,NaN,NaN,57.349261,42.499091,20.0,20.0,50.0,70.0,190.0
MS Zoning,object,0,0.0,155728,7,0.3,2637,7,RL,2043,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Lot Frontage,float64,449,17.0,21096,126,4.8,2188,NaN,NaN,NaN,69.166819,23.356779,21.0,58.0,68.0,80.0,313.0
Lot Area,int64,0,0.0,21096,1799,68.2,2637,NaN,NaN,NaN,10044.694729,6742.549521,1300.0,7436.0,9450.0,11526.0,164660.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Mo Sold,int64,0,0.0,21096,12,0.5,2637,NaN,NaN,NaN,6.243838,2.722093,1.0,4.0,6.0,8.0,12.0
Yr Sold,int64,0,0.0,21096,5,0.2,2637,NaN,NaN,NaN,2007.795601,1.306403,2006.0,2007.0,2008.0,2009.0,2010.0
Sale Type,object,0,0.0,158288,10,0.4,2637,10,WD,2286,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sale Condition,object,0,0.0,166537,6,0.2,2637,6,Normal,2166,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Redo as a Function

In [ ]:
def metadata( dataframe ):
  '''Given a dataframe, returns a dataframe of metadata about the dataframe'''
  metadata_df = pd.DataFrame()
  metadata_df["Data_types"] = dataframe.dtypes
  metadata_df["Count"] = len(dataframe)
  metadata_df["Nulls"] = dataframe.isnull().sum()
  metadata_df["Nulls_pct"] = ( metadata_df["Nulls"] / metadata_df["Count"] * 100 ).round(1)
  metadata_df["Memory"] = dataframe.memory_usage( deep = True)
  metadata_df["NUnique"] = dataframe.nunique()
  metadata_df["NUnique_pct"] = (dataframe.nunique() / metadata_df["Count"] * 100).round(1)
  metadata_df = metadata_df.join( dataframe.describe( include = "all" ).transpose() )
  metadata_df["IRQ"] = metadata_df["75%"] - metadata_df["25%"]
  metadata_df["range"] = metadata_df["max"] - metadata_df["min"]
  metadata_df["sum"] = metadata_df["mean"] * metadata_df["count"]
  metadata_df = (
    metadata_df
    .astype( { "count" : int } )
    .rename( columns = {
      "25%" : "Q1_25%",
      "50%" : "Q2_median",
      "75%" : "Q3_75%",
      }
    )
  )
  return metadata_df


In [ ]:
md = metadata(housing)
md

,Data_types,Count,Nulls,Nulls_pct,Memory,NUnique,NUnique_pct,count,unique,top,...,mean,std,min,Q1_25%,Q2_median,Q3_75%,max,IRQ,range,sum
PID,int64,2637,0,0.0,21096,2637,100.0,2637,NaN,NaN,...,714130147.70383,188752674.750322,526301100.0,528477010.0,535453040.0,907187010.0,1007100110.0,378710000.0,480799010.0,1883161199495.0
MS SubClass,int64,2637,0,0.0,21096,16,0.6,2637,NaN,NaN,...,57.349261,42.499091,20.0,20.0,50.0,70.0,190.0,50.0,170.0,151230.0
MS Zoning,object,2637,0,0.0,155728,7,0.3,2637,7,RL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Lot Frontage,float64,2637,449,17.0,21096,126,4.8,2188,NaN,NaN,...,69.166819,23.356779,21.0,58.0,68.0,80.0,313.0,22.0,292.0,151337.0
Lot Area,int64,2637,0,0.0,21096,1799,68.2,2637,NaN,NaN,...,10044.694729,6742.549521,1300.0,7436.0,9450.0,11526.0,164660.0,4090.0,163360.0,26487860.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Mo Sold,int64,2637,0,0.0,21096,12,0.5,2637,NaN,NaN,...,6.243838,2.722093,1.0,4.0,6.0,8.0,12.0,4.0,11.0,16465.0
Yr Sold,int64,2637,0,0.0,21096,5,0.2,2637,NaN,NaN,...,2007.795601,1.306403,2006.0,2007.0,2008.0,2009.0,2010.0,2.0,4.0,5294557.0
Sale Type,object,2637,0,0.0,158288,10,0.4,2637,10,WD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sale Condition,object,2637,0,0.0,166537,6,0.2,2637,6,Normal,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


A count of columns with each data type.

In [ ]:
md["Data_types"].value_counts()

,count
Data_types,
object,43
int64,27
float64,11


All columns with more than some threshold of nulls.

In [ ]:
filter = ( md["Nulls_pct"] > 10 )
md["Nulls_pct"][ filter ].sort_values( ascending = False )

,Nulls_pct
Pool QC,99.6
Misc Feature,96.4
Alley,93.2
Fence,80.0
Mas Vnr Type,60.9
Fireplace Qu,48.7
Lot Frontage,17.0


Identifier columns

In [ ]:
filter = ( md["NUnique_pct"] > 95 )
md[["NUnique","NUnique_pct"]][ filter ].sort_values( by = ["NUnique_pct"], ascending = False )

,NUnique,NUnique_pct
PID,2637,100.0


Skew


In [ ]:
md["mean"] > md["Q2_median"]

,0
PID,True
MS SubClass,True
MS Zoning,False
Lot Frontage,True
Lot Area,True
...,...
Mo Sold,True
Yr Sold,False
Sale Type,False
Sale Condition,False


In [ ]:
non=md["mean"] > md["Q2_median"]
non

,0
PID,True
MS SubClass,True
MS Zoning,False
Lot Frontage,True
Lot Area,True
...,...
Mo Sold,True
Yr Sold,False
Sale Type,False
Sale Condition,False


In [ ]:
non [non>0]

,0
PID,True
MS SubClass,True
Lot Frontage,True
Lot Area,True
Overall Qual,True
Overall Cond,True
Mas Vnr Area,True
BsmtFin SF 1,True
BsmtFin SF 2,True
Bsmt Unf SF,True


In [ ]:
## Write package to python for later import

%%writefile metadata.py
import pandas as pd

def metadata( dataframe ):
  '''Given a dataframe, returns a dataframe of metadata about the dataframe'''
  metadata_df = pd.DataFrame()
  metadata_df["Data_types"] = dataframe.dtypes
  metadata_df["Count"] = len(dataframe)
  metadata_df["Nulls"] = dataframe.isnull().sum()
  metadata_df["Nulls_pct"] = ( metadata_df["Nulls"] / metadata_df["Count"] * 100 ).round(1)
  metadata_df["Memory"] = dataframe.memory_usage( deep = True)
  metadata_df["NUnique"] = dataframe.nunique()
  metadata_df["NUnique_pct"] = (dataframe.nunique() / metadata_df["Count"] * 100).round(1)
  metadata_df = metadata_df.join( dataframe.describe( include = "all" ).transpose() )
  metadata_df = metadata_df.astype( { "count" : int } )
  if dataframe.select_dtypes(include=['number']).shape[1] :
    metadata_df["IRQ"] = metadata_df["75%"] - metadata_df["25%"]
    metadata_df["range"] = metadata_df["max"] - metadata_df["min"]
    metadata_df["sum"] = metadata_df["mean"] * metadata_df["count"]
    metadata_df = (
      metadata_df
      .rename( columns = {
        "25%" : "Q1_25%",
        "50%" : "Q2_median",
        "75%" : "Q3_75%",
        }
      )
    )
  return metadata_df

def cols_to_drop( dataframe ):
  '''Given a dataframe, returns columns that should likely be dropped'''
  md = metadata( dataframe )
  filter = md["Nulls_pct"] >= 40
  return md[ filter ]["Nulls_pct"].to_dict()

def likely_ids( dataframe ):
  '''Given a dataframe, returns a dictionary of likely ID columns'''
  md = metadata( dataframe )
  return md["NUnique_pct"][ md["NUnique_pct"] > 95 ].sort_values( ascending = False )


Writing metadata.py


In [ ]:
!ls -la m*


-rw-r--r-- 1 root root 1610 Jun 18 03:35 metadata.py


In [ ]:
!cat -n metadata.py


     1	import pandas as pd
     2	
     3	def metadata( dataframe ):
     4	  '''Given a dataframe, returns a dataframe of metadata about the dataframe'''
     5	  metadata_df = pd.DataFrame()
     6	  metadata_df["Data_types"] = dataframe.dtypes
     7	  metadata_df["Count"] = len(dataframe)
     8	  metadata_df["Nulls"] = dataframe.isnull().sum()
     9	  metadata_df["Nulls_pct"] = ( metadata_df["Nulls"] / metadata_df["Count"] * 100 ).round(1)
    10	  metadata_df["Memory"] = dataframe.memory_usage( deep = True)
    11	  metadata_df["NUnique"] = dataframe.nunique()
    12	  metadata_df["NUnique_pct"] = (dataframe.nunique() / metadata_df["Count"] * 100).round(1)
    13	  metadata_df = metadata_df.join( dataframe.describe( include = "all" ).transpose() )
    14	  metadata_df = metadata_df.astype( { "count" : int } )
    15	  if dataframe.select_dtypes(include=['number']).shape[1] :
    16	    metadata_df["IRQ"] = metadata_df["75%"] - metadata_df["25%"]
    17	    metadata_df["range"] =

In [ ]:
# find nulls, list columns with nulls, sort from lowest to highest count of nulls
## non=housing_clean.isnull().sum().sort_values()
## non [non>0]

In [ ]:
# from Metadata Lecture
# how can/did you discover if there are any nulls and how would/did you handle them?
nulls = housing_clean.isnull().sum()
filter = nulls > 0
( nulls[ filter ].sort_values( ascending = False ) / housing.shape[0] * 100 ).round(1)

In [ ]:
# how can/did you discover if there are any nulls and how would/did you handle them?

In [ ]:
# Look at CarName column
cars['CarName'].value_counts()

In [ ]:
# Keep just the name of the car manufacturer
car_split = lambda x: x.split(" ")[0]

def car_split(x):
  return x.split(" ")[0]

cars['carCompany'] = cars['CarName'].apply(lambda x: x.split(" ")[0])

# pd.Series([ x.split(" ")[0] for x in cars['CarName'] ][:10])

# cars['CarName'].apply(lambda x: x.split(" ")[0])


In [ ]:
cars['carCompany'].value_counts()

In [ ]:
# # Fix spelling
# cars['carCompany'] = cars['carCompany'].str.replace('vw', 'volkswagen')
# cars['carCompany'] = cars['carCompany'].str.replace('vokswagen', 'volkswagen')
# cars['carCompany'] = cars['carCompany'].str.replace('porcshce', 'porsche')
# cars['carCompany'] = cars['carCompany'].str.replace('maxda', 'mazda')
# cars['carCompany'] = cars['carCompany'].str.replace('toyouta', 'toyota')
# cars['carCompany'] = cars['carCompany'].str.replace('Nissan', 'nissan')


In [ ]:
# create mapping between incorrect and correct spelling
spelling_dict={
  'alfa-romero': 'alfa-romeo',
  'maxda': 'mazda',
  'Nissan': 'nissan',
  'porcshce': 'porsche',
  'toyouta': 'toyota',
  'vw': 'volkswagen',
  'vokswagen': 'volkswagen',
}

In [ ]:
# Fix spelling
for k, v in spelling_dict.items():
  cars['carCompany'] = cars['carCompany'].str.replace( k, v )

cars['carCompany'].value_counts()


In [ ]:
# Drop carname column
cars.drop('CarName', axis = 1, inplace = True)

In [ ]:
cars.head()

In [ ]:
cars['doornumber'].value_counts(), cars['cylindernumber'].value_counts()

In [ ]:
# Change doornumber and cylindernumber to numeric form
cars['doornumber'] = cars['doornumber'].replace({
    'four': "4",
    'two':  "2",
    }).astype(int)
cars['cylindernumber'] = cars['cylindernumber'].replace({
    'two':    "2",
    'three':  "3",
    'four':   "4",
    'five':   "5",
    'six':    "6",
    'eight':  "8",
    'twelve': "12",
    }).astype(int)
cars.head()

In [ ]:
cars.dtypes.sort_values()

In [ ]:
# Pull out object data types
cars_categorical = cars.select_dtypes(include = 'object')
cars_categorical.head()

In [ ]:
# One hot encode these categories
cars_encoded = pd.get_dummies(cars_categorical)
cars_encoded.head()

In [ ]:
cars_encoded.shape

In [ ]:
cars_categorical.shape

In [ ]:
# Combine encoded dataframe to data frame with just numeric values
cars_numerical = cars.select_dtypes(include = ['int64', 'float64'])
cars_numerical.shape

In [ ]:
cars_combined = pd.concat([cars_numerical, cars_encoded], axis = 1)
cars_combined.head()

In [ ]:
cars_combined.shape

In [ ]:
cars_combined.dtypes.value_counts()

## Processing

First, we will fit a model with all predictors.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split


In [ ]:
# split into Target and Features
X = cars_combined.drop('price', axis = 1)
y = cars_combined['price']

In [ ]:
# Perform CV
n = 500
results = np.zeros(n)
for idx in range(n):
  X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25)
  model = LinearRegression()
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  results[idx] = np.sqrt(mean_squared_error(y_test,y_pred))
print(f"CV RMSE: {results.mean().round(2)}")
print(f"Number of Predictors: {len(X.columns)}")

## Feature Selection

## Filter Methods
With filter methods, we will calculate a statistical metric for each column. Based on that metric, it will be decided whether that feature will be kept or removed from our predictive model. *Note*: the choice of statistical metric depends on the data type of the feature.

**Pros**: Easy to implement

**Cons**: Does not account for interactions between features

Image from [Toward Data Science:](https://towardsdatascience.com/learn-how-to-do-feature-selection-the-right-way-61bca8557bef)

![An image](https://miro.medium.com/max/1400/1*tzfWABEHK9-4SOaSl1mdRA.png)


### Numerical Predictors & Numerical Response - Pearson Correlation Coefficient

In [ ]:
# Look at numerical predictors
cars_numerical.head()

In [ ]:
# Sort correlations w/ price
cars_numerical.corr()['price'].abs().sort_values(ascending = False)

In [ ]:
# Plot result
cars_numerical.corr()['price'].abs().sort_values(ascending = False).plot(kind = 'bar', figsize = (10,5)) ;

WARNING: The correlation coefficient only measures the strength of the linear relationship between two variables. If the relationship is nonlinear, it may not be reflected in the correlation coefficient.

Example from [Toward Data Science:](https://towardsdatascience.com/learn-how-to-do-feature-selection-the-right-way-61bca8557bef)
![An image](https://miro.medium.com/max/1400/1*cY24YPkTGmuzaSaUmdTbmw.png)

In [ ]:
# Fit linear regression model w/ just continuous predictors highly correlated with response
corrs = cars_numerical.corr()['price'].abs().sort_values(ascending = False)
keep = corrs[(corrs>.2) & (corrs <1)]
keep

In [ ]:
X_corr = X[keep.index]
X_corr.head()

In [ ]:
# Combine correlated numerical predictors with all encoded categorical predictors
X = pd.concat([X_corr, cars_encoded], axis = 1)
X.head()

In [ ]:
X.shape

In [ ]:
# Perform CV
n = 500
results = np.zeros(n)
for idx in range(n):
  X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25)
  model = LinearRegression()
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  results[idx] = np.sqrt(mean_squared_error(y_test,y_pred))
print(f"CV RMSE: {results.mean().round(2)}")
print(f"Number of Predictors: {len(X.columns)}")

### ANOVA - Categorical Predictors & Numerical Response
ANOVA works by testing if the means of different categories are statistically different. For example, it would test to see if the mean price for BMWs is statistically different from the mean price of Buicks.

It is based on the following two hypotheses
H0: Means of all groups are equal. (null hypothesis)
H1: At least one mean of the groups are different.

ANOVA calculates an F-Test for statistical signficance. The larger the value of the F-test, the more likely we are to reject the null hypothesis.

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

fvalue_selector = SelectKBest(f_regression, k='all')  #select features best ANOVA F-Values
fvalue_selector.fit_transform(cars_encoded, y)

# Plot results
results = pd.DataFrame({'ANOVA': fvalue_selector.scores_}, index = cars_encoded.columns)
results.sort_values(by = 'ANOVA', ascending = False).plot(kind = 'bar', figsize = (15,5))

In [ ]:
# Just keep the results with F-values greater than 20
anova = results.sort_values(by = 'ANOVA', ascending = False)
keep = anova['ANOVA'][anova['ANOVA'] > 20]
keep

In [ ]:
X_anova = X[keep.index]

In [ ]:
X_anova

In [ ]:
X2 = pd.concat([X_anova, X_corr], axis = 1)
X2.head()

In [ ]:
( X_corr.shape, X_anova.shape, X.shape, X2.shape)

In [ ]:
# Perform CV
n = 500
rmse_results = np.zeros(n)
for idx in range(n):
  X_train, X_test, y_train, y_test = train_test_split(X2,y,test_size=0.25)
  model = LinearRegression()
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  rmse_results[idx] = np.sqrt(mean_squared_error(y_test,y_pred))
print(f"CV RMSE: {rmse_results.mean().round(2)}")
print(f"Number of Predictors: {len(X2.columns)}")

## Regularization

In [ ]:
from sklearn.linear_model import Lasso

Scale the data frame using the Standard Scaler

In [ ]:
cars_combined

In [ ]:
cars_combined_bak = cars_combined.copy()
cars_combined_bak.shape

In [ ]:
# restore
cars_combined = cars_combined_bak.copy()


Use standard scaler to scale data set.

This data set is the cleaned without the feature selection from correlation.

In [ ]:
# %%capture --no-stdout
X_lasso = cars_combined.drop('price', axis = 1)
y_lasso = cars_combined['price']

# Specify values of alpha we want to try
alph = [50, 75, 100, 200, 300, ]

# Create empty list to hold results
results_alpha = []

for i in alph:
  # Calculate CV RMSE
  n = 200
  results = np.zeros(n)

  for idx in range(n):
    # Create the model
    my_fit_lasso = Lasso(alpha = i, tol = .01, max_iter=10_000); # Looping through alpha

    # Train, test split
    X_train, X_test, y_train, y_test = train_test_split(X_lasso, y_lasso, test_size=0.25)

    # Standard scale only training features
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)

    # Scale testing features
    X_test = scaler.transform(X_test)

    # Fit regression
    my_fit_lasso.fit(X_train, y_train)

    # Predict
    y_pred = my_fit_lasso.predict(X_test)

    # Calculate and record performance metric
    results[idx] = np.sqrt(mean_squared_error(y_test,y_pred))

  results_alpha.append(results.mean())

results_df = pd.DataFrame({'Alpha': alph, 'RMSE': results_alpha})


In [ ]:
results_df.sort_values( by = 'RMSE' )


Get the Alpha that results in the lowest RMSE.

In [ ]:
results_df.sort_values( by = 'RMSE' )["Alpha"][0]


In [ ]:
results_df.sort_values( by = 'RMSE' )["Alpha"].iloc[0]


In [ ]:
alpha = results_df.sort_values( by = 'RMSE' )["Alpha"].iloc[0]
alpha

In [ ]:
# %%capture --no-stdout
# Perform CV
n = 500
results = np.zeros(n)

for idx in range(n):
  # Train, test split
  X_train, X_test, y_train, y_test = train_test_split(X_lasso, y_lasso, test_size=0.25)

  # Standard scale only training features
  scaler = StandardScaler()
  scaler.fit(X_train)
  X_train = scaler.transform(X_train)

  # Scale testing features
  X_test = scaler.transform(X_test)

  # Create the model
  model = Lasso(alpha = alpha, tol = .01, max_iter=10000)

  # Fit regression
  model.fit(X_train, y_train)

  # Predict
  y_pred = model.predict(X_test)

  # Calculate and record performance metric
  results[idx] = np.sqrt(mean_squared_error(y_test,y_pred))

print(f"CV RMSE: {results.mean().round(2)}")
print(f"Number of Predictors: {sum(model.coef_ >0)}")

In [ ]:
X_lasso.shape

In [ ]:
len(X_lasso.columns)

In [ ]:
# Plot model coefficients
model_results = pd.DataFrame()
model_results['Predictor'] = X_lasso.columns
model_results['Lasso'] = model.coef_
model_results.sort_values( by = ["Lasso"], ascending = False ).plot(
    x = 'Predictor',
    kind = 'bar',
    figsize = (20,5)
    )
plt.grid()

In [ ]:
model_results['Lasso'].abs().sort_values()

In [ ]:
# Plot parameters with coefficients greater than 0
greater_0 = model_results[(model_results['Lasso'] > 500 ) | (model_results['Lasso'] < -500 )]
greater_0.sort_values( by = ["Lasso"], ascending = False ).plot(x = 'Predictor', kind = 'bar', figsize = (10,5))
plt.grid()